# Installating R on WinPython

#### This procedure applys for Winpython  (Version of December 2015 and after) 
### 1 - Downloading R binary

In [2]:
import os
import sys
import io

In [3]:
# downloading R may takes a few minutes (80Mo)
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2

In [4]:
# specify R binary and (md5, sha1) hash
# R-3.4.3:
r_url = "https://cran.r-project.org/bin/windows/base/R-3.4.3-win.exe"
hashes=("0ff087acbae677d7255af19b0a9df27f","aabf0b671ae1dca741c3df9dee976a7d4b584f80")

In [7]:
# specify target location
r_installer = os.environ["WINPYDIR"]+"\\..\\tools\\"+os.path.basename(r_url)
os.environ["r_installer"] = r_installer

print(r_installer)
print(urllib2)

D:\WinPython\python-3.5.4.amd64\..\tools\R-3.4.3-win.exe
<module 'urllib.request' from 'D:\\WinPython\\python-3.5.4.amd64\\lib\\urllib\\request.py'>


In [6]:
# Download
g = urllib2.urlopen(r_url) 
# Above Command is failing
with io.open(r_installer, 'wb') as f:
    f.write(g.read())
g.close
g = None

#checking it's there
!dir %r_installer%

HTTPError: HTTP Error 404: Not Found


### 2 - checking and Installing R binary in the right place

In [7]:
# checking it's the official R
import hashlib
def give_hash(of_file, with_this):
    with io.open(r_installer, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*12+"MD5"+" "*(32-12-3)+" "+" "*15+"SHA-1"+" "*(40-15-5)+"\n"+"-"*32+" "+"-"*40)
print ("%s %s %s" % (give_hash(r_installer, hashlib.md5) , give_hash(r_installer, hashlib.sha1),r_installer))
if give_hash(r_installer, hashlib.md5) == hashes[0] and give_hash(r_installer, hashlib.sha1) == hashes[1]:
   print("looks good!")
else:
   print("problem ! please check")
   assert give_hash(r_installer, hashlib.md5) == hashes[0]
   assert give_hash(r_installer, hashlib.sha1) == hashes[1]

            MD5                                 SHA-1                    
-------------------------------- ----------------------------------------
0ff087acbae677d7255af19b0a9df27f aabf0b671ae1dca741c3df9dee976a7d4b584f80 C:\WinPython\basedir36\buildQt5\winpython-64bit-3.6.x.0\python-3.6.4.amd64\..\tools\R-3.4.3-win.exe
looks good!


In [8]:
# preparing Dos variables
os.environ["R_HOME"] = os.environ["WINPYDIR"]+ "\\..\\tools\\R\\" 
os.environ["R_HOMEbin"]=os.environ["R_HOME"] + "bin" 

# for installation we need this
os.environ["tmp_Rbase"]=os.path.join(os.path.split(os.environ["WINPYDIR"])[0]  , 'tools','R' )  
if 'amd64' in sys.version.lower():
    r_comp ='/COMPONENTS="main,x64,translations'
else:
    r_comp ='/COMPONENTS="main,i386,translations'
os.environ["tmp_R_comp"]=r_comp


In [9]:
# let's install it, if hashes do match
assert give_hash(r_installer, hashlib.md5) == hashes[0]
assert give_hash(r_installer, hashlib.sha1) == hashes[1]
# If you are "USB life style", or multi-winpython
#   ==> CLICK the OPTION "Don't create a StartMenuFolder' <== (when it will show up)

!start cmd /C %r_installer% /DIR=%tmp_Rbase% %tmp_R_comp%

## During Installation (if you wan't to move the R installation after)

Choose non default option "Yes (customized startup"

then after 3 screens, Select  "Don't create a Start Menu Folder"

Un-select "Create a desktop icon"

Un-select "Save version number in registery"


<img src="https://raw.githubusercontent.com/stonebig/winpython_afterdoc/master/examples/images/r_setup_unclick_shortcut.GIF">

### 3 - create a R_launcher and install irkernel

In [10]:
import os
import sys
import io
# let's create a R launcher  
r_launcher = r"""
@echo off
call %~dp0env.bat
rscript %*
"""
r_launcher_bat = os.environ["WINPYDIR"]+"\\..\\scripts\\R_launcher.bat"

# let's create a R init script
# in manual command line, you can use repos = c('http://irkernel.github.io/', getOption('repos'))
r_initialization = r"""
install.packages(c('repr', 'IRdisplay', 'stringr', 'crayon', 'pbdZMQ', 'devtools'), repos = c('http://cran.rstudio.com/', 'http://cran.rstudio.com/'))
devtools::install_github('IRkernel/IRkernel')
library('pbdZMQ')
library('repr')
library('IRkernel')
library('IRdisplay')
library('crayon')
library('stringr')
IRkernel::installspec()
"""
r_initialization_r = os.path.normpath(os.environ["WINPYDIR"]+"\\..\\scripts\\R_initialization.r")


for i in [(r_launcher,r_launcher_bat), (r_initialization, r_initialization_r)]:
    with io.open(i[1], 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in i[0].splitlines():
            f.write('%s\n' %  line  )


In [11]:
#check what we are going to do 
print ("!start cmd /C %WINPYDIR%\\..\\scripts\\R_launcher.bat --no-restore --no-save  " + r_initialization_r)

!start cmd /C %WINPYDIR%\..\scripts\R_launcher.bat --no-restore --no-save  C:\WinPython\basedir36\buildQt5\winpython-64bit-3.6.x.0\scripts\R_initialization.r


In [12]:
# Launch Rkernel setup
os.environ["r_initialization_r"] = r_initialization_r
!start cmd /C %WINPYDIR%\\..\\scripts\\R_launcher.bat --no-restore --no-save  %r_initialization_r%  

In [13]:
# make RKernel a movable installation with the rest of WinPython 
from winpython import utils
base_winpython = os.path.dirname(os.path.normpath(os.environ["WINPYDIR"]))
rkernel_json=(base_winpython+"\\settings\\kernels\\ir\\kernel.json")

# so we get "argv": ["{prefix}/../tools/R/bin/x64/R"
utils.patch_sourcefile(rkernel_json, base_winpython.replace("\\","/"), r'{prefix}/..', silent_mode=False)

patching  C:\WinPython\basedir36\buildQt5\winpython-64bit-3.6.x.0\settings\kernels\ir\kernel.json from C:/WinPython/basedir36/buildQt5/winpython-64bit-3.6.x.0 to {prefix}/..


### 4- Install a R package via a IPython  Kernel

In [14]:
%load_ext rpy2.ipython

#vitals: 'dplyr', 'R.utils', 'nycflights13'
# installation takes 2 minutes
%R install.packages(c('dplyr','R.utils', 'nycflights13'), repos='http://cran.rstudio.com/') 

### 5- Small demo via R magic

In [15]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [16]:
%%R
library('dplyr')
library('nycflights13') 
write.csv(flights, "flights.csv")

In [17]:
%R head(flights) 

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
1,2013,1,1,517,515,2.0,830,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400.0,5.0,15.0,1.357016e+09
2,2013,1,1,533,529,4.0,850,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416.0,5.0,29.0,1.357016e+09
3,2013,1,1,542,540,2.0,923,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089.0,5.0,40.0,1.357016e+09
4,2013,1,1,544,545,-1.0,1004,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576.0,5.0,45.0,1.357016e+09
5,2013,1,1,554,600,-6.0,812,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762.0,6.0,0.0,1.357020e+09
6,2013,1,1,554,558,-4.0,740,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719.0,5.0,58.0,1.357016e+09


In [18]:
%R airports %>% mutate(dest = faa) %>% semi_join(flights) %>% head

,faa,name,lat,lon,alt,tz,dst,tzone,dest
1,ABQ,Albuquerque International Sunport,35.040222,-106.609194,5355,-7.0,A,America/Denver,ABQ
2,ACK,Nantucket Mem,41.253053,-70.060181,48,-5.0,A,America/New_York,ACK
3,ALB,Albany Intl,42.748267,-73.801692,285,-5.0,A,America/New_York,ALB
4,ANC,Ted Stevens Anchorage Intl,61.174361,-149.996361,152,-9.0,A,America/Anchorage,ANC
5,ATL,Hartsfield Jackson Atlanta Intl,33.636719,-84.428067,1026,-5.0,A,America/New_York,ATL
6,AUS,Austin Bergstrom Intl,30.194528,-97.669889,542,-6.0,A,America/Chicago,AUS


### 6 - Installing the very best of R pakages (optional, you will start to get a really big directory)


In [19]:
# essentials: 'tidyr', 'shiny', 'ggplot2', 'caret' , 'nnet'  
# remaining of Hadley Wickahm "stack" (https://github.com/rstudio)
%R install.packages(c('tidyr', 'ggplot2',  'shiny','caret' , 'nnet'), repos='https://cran.rstudio.com/') 
%R install.packages(c('knitr', 'purrr', 'readr', 'readxl'), repos='https://cran.rstudio.com/')
%R install.packages(c('rvest', 'lubridate', 'ggvis', 'readr','base64enc'), repos='https://cran.rstudio.com/')

# TRAINING = online training book http://r4ds.had.co.nz/ (or https://github.com/hadley/r4ds)

### 7 - Relaunch Jupyter Notebook to get a R kernel option
launch a new notebook of "R" type, and type in it:
    
library('dplyr')

library('nycflights13') 

head(flights)

### 9 - To Un-install  / Re-install R (or other trouble-shooting)

- launch  winpython**\tools\R\unins000.exe

- delete the directory winpython**\tools\R

- re-install
